In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

In [11]:
df = pd.read_csv("../data/cleaned_data.csv")
df.head()

,id,building_type,building_subtype,price,living_area,frontages,bedrooms,bathrooms,liv_room_surf,kitchen_type,...,swimming_pool,status_build,furnished,energy_class,heating,flood_zone,surroundings,zipcode,municipality,province
0,10567498,apartment,apartment,730000.0,212.0,3.0,4.0,2.0,60.0,1.0,...,0.0,2.0,0.0,2.0,Gas,NaN,Isolated,1050,Ixelles,Brussels Capital Region
1,10567435,apartment,loft,775000.0,258.0,2.0,2.0,1.0,NaN,3.0,...,NaN,2.0,1.0,7.0,Gas,Non flood zone,NaN,3020,Winksele,Flemish Brabant
2,10557731,apartment,apartment,99000.0,94.0,4.0,2.0,1.0,34.0,1.0,...,0.0,2.0,0.0,3.0,Gas,Non flood zone,Urban,4820,Dison,Liège
3,10567501,apartment,apartment,730000.0,212.0,3.0,4.0,2.0,60.0,1.0,...,0.0,2.0,0.0,2.0,Gas,NaN,Isolated,1060,Saint-Gilles,Brussels Capital Region
4,10557727,apartment,apartment,219000.0,111.0,2.0,3.0,1.0,30.0,1.0,...,0.0,2.0,0.0,6.0,Gas,Non flood zone,Isolated,4680,Oupeye,Liège


In [12]:
df.shape

(9341, 26)

In [13]:
missing_values = df.isnull().sum()
missing_values

id                     0
building_type          0
building_subtype       0
price                 37
living_area          132
frontages           1678
bedrooms             152
bathrooms           1204
liv_room_surf       3616
kitchen_type        1039
showers             6197
toilets              831
basement            4848
terrace                0
garden                 0
elevator            4203
swimming_pool       5377
status_build        1054
furnished           4288
energy_class           0
heating             1261
flood_zone          4240
surroundings        5330
zipcode                0
municipality           0
province               0
dtype: int64

In [14]:
df = df.drop(columns = ["kitchen_type", "status_build","bedrooms", "liv_room_surf", "frontages", "bathrooms", "id", "basement","elevator", "surroundings","flood_zone", "heating", "showers","toilets", "furnished", "heating","flood_zone"])

#dropna living_area, price
df = df.dropna(subset=["living_area", "price"])

#replace swimming_pool None with 0
df["swimming_pool"] = df["swimming_pool"].fillna(0)

In [15]:
missing_values = df.isnull().sum()
missing_values

building_type       0
building_subtype    0
price               0
living_area         0
terrace             0
garden              0
swimming_pool       0
energy_class        0
zipcode             0
municipality        0
province            0
dtype: int64

In [16]:
df.shape

(9172, 11)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9172 entries, 0 to 9340
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   building_type     9172 non-null   object 
 1   building_subtype  9172 non-null   object 
 2   price             9172 non-null   float64
 3   living_area       9172 non-null   float64
 4   terrace           9172 non-null   float64
 5   garden            9172 non-null   float64
 6   swimming_pool     9172 non-null   float64
 7   energy_class      9172 non-null   float64
 8   zipcode           9172 non-null   int64  
 9   municipality      9172 non-null   object 
 10  province          9172 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 859.9+ KB


In [19]:
from sklearn.preprocessing import OneHotEncoder
 
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',
      len(object_cols))

Categorical variables:
['building_type', 'building_subtype', 'municipality', 'province']
No. of. categorical features:  4
